# Friday 3.2
let's try this again but rip off ChatGPT

## Imports

In [1]:
import re
from transformers import AutoTokenizer, AutoModelForCausalLM,  AutoModelForSeq2SeqLM
from transformers import Trainer, TrainingArguments
from datasets import load_dataset
import torch

2023-01-24 22:21:22.113888: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-24 22:21:22.113922: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-24 22:21:23.029007: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-01-24 22:21:23.029121: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-01-24 22:21:23.029136: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT War

## Load base model

In [2]:
# Set the model and tokenizer names
model_name = "bigscience/T0_3B"
tokenizer_name = "bigscience/T0_3B"

# Load the model and tokenizer
model =  AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
#tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-125M")
#model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-neo-125M")

Downloading:   0%|          | 0.00/632 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/11.4G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.86k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

## Level fine tunning
Do a statndard fine tune on the model

### The finetune function

In [3]:
def fine_tune(model, train_dataloader, val_dataloader, epochs=2):
  # Set up the trainer
  training_args = TrainingArguments(
      output_dir="./results",          # output directory
      num_train_epochs=epochs,         # number of epochs
      per_device_train_batch_size=16,  # batch size
      per_device_eval_batch_size=64,   # evaluation batch size
      warmup_steps=500,                # number of warmup steps
      weight_decay=0.01,               # L2 weight decay
      logging_dir="./logs"             # logging directory
  )

  trainer = Trainer(
      model=model,                     # the model to train
      args=training_args,              # training arguments
      train_dataset=train_dataloader,  # training dataset
      eval_dataset=val_dataloader      # validation dataset
  )

  # Fine-tune the model
  trainer.train()

### Data loader

In [4]:
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
dataset = load_dataset("Anthropic/hh-rlhf")
def tokenize_function(examples):
    return tokenizer(examples["chosen"], padding="max_length", truncation=True)
tokenized_datasets = dataset.map(tokenize_function, batched=True)


Using custom data configuration Anthropic--hh-rlhf-c8cd8dc58ab67414


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /home/jovyan/.cache/huggingface/datasets/Anthropic___json/Anthropic--hh-rlhf-c8cd8dc58ab67414/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/161 [00:00<?, ?ba/s]

  0%|          | 0/9 [00:00<?, ?ba/s]

In [5]:
train_dataloader = tokenized_datasets["train"].shuffle(seed=42)
val_dataloader = tokenized_datasets["test"].shuffle(seed=42)

### Tune it!

In [6]:
fine_tune(model, train_dataloader, val_dataloader)

The following columns in the training set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: chosen, rejected. If chosen, rejected are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 160800
  Num Epochs = 2
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 20100
  Number of trainable parameters = 2849757184


: 

: 

In [ ]:
def generate_response_single(prompt):
  # Encode the prompt and generate a response
  input_ids = tokenizer.encode(prompt, return_tensors='pt')
  response = model.generate(input_ids, max_length=1024, top_p=0.9, top_k=20)
  print(tokenizer.decode(response[0]))
generate_response_single("Human: Hello there Assistant: ")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Human: Hello there Assistant: 

I am a newbie in the field of programming and I am trying to understand the basics of programming. I am new to programming and I am trying to understand the basics of programming. I am new to programming and I am trying to understand the basics of programming. I am new to programming and I am trying to understand the basics of programming. I am new to programming and I am trying to understand the basics of programming. I am new to programming and I am trying to understand the basics of programming. I am new to programming and I am trying to understand the basics of programming. I am new to programming and I am trying to understand the basics of programming. I am new to programming and I am trying to understand the basics of programming. I am new to programming and I am trying to understand the basics of programming. I am new to programming and I am trying to understand the basics of programming. I am new to programming and I am trying to understand the b

## Chatbot interface

In [ ]:
def chatbot():
  step = 0
  chat_history = []
  while(True):
    new_user_input_ids = tokenizer.encode(input(">> User:") + tokenizer.eos_token, return_tensors='pt')

    # append the new user input tokens to the chat history
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

    # generated a response while limiting the total chat history to 1000 tokens, 
    chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)

    # pretty print last ouput tokens from bot
    print("Friday: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))
    step = step + 1

# Run the chatbot
chatbot()

Friday: Q:

How to get the value of a variable in a function?

I have a function that takes a variable and returns a value.
function get_value(var) {
    var value = 0;
    return value;
}

I want to get the value of the variable in the function.
function get_value(var) {
    var value = 0;
    return value;
}

I have tried to use the function like this:
function get_value(var) {
    var value = 0;
    return value;
}

But it doesn't work.

A:

You can use the function like this:
function get_value(var) {
    var value = 0;
    return value;
}

function get_value(var) {
    var value = 0;
    return value;
}

function get_value(var) {
    var value = 0;
    return value;
}

function get_value(var) {
    var value = 0;
    return value;
}

function get_value(var) {
    var value = 0;
    return value;
}

function get_value(var) {
    var value = 0;
    return value;
}

function get_value(var) {
    var value = 0;
    return value;
}

function get_value(var) {
    var value = 0;
    retu

Input length of input_ids is 1001, but ``max_length`` is set to 1000. This can lead to unexpected behavior. You should consider increasing ``config.max_length`` or ``max_length``.


KeyboardInterrupt: 